In [6]:
import pymongo
import pandas
from pymongo import MongoClient
client = MongoClient('mongodb://xx:xx@xx/drugbank')
db = client.drugbank

In [16]:
db.collection_names()

['drugs', 'drugs_sub', 'drugs_int', 'drugs_unnst', 'drugs_targets']

In [18]:
drugs_targets = db.drugs_targets

In [44]:
data_targets = drugs_targets.find({},{'_id':0})
import pandas as pd
df = pd.DataFrame(list(data_targets))
df.head()

,category,class,drug_id,drug_name,kingdom,parent,subclass,superclass,target
0,"Amino Acids, Peptides, and Proteins",Carboxylic Acids and Derivatives,DB00001,Lepirudin,Organic Compounds,Peptides,"Amino Acids, Peptides, and Analogues",Organic Acids,"[BE0000048_inhibitor, BE0000048, Prothrombin, ..."
1,Anti-coagulant,Carboxylic Acids and Derivatives,DB00001,Lepirudin,Organic Compounds,Peptides,"Amino Acids, Peptides, and Analogues",Organic Acids,"[BE0000048_inhibitor, BE0000048, Prothrombin, ..."
2,Anticoagulants,Carboxylic Acids and Derivatives,DB00001,Lepirudin,Organic Compounds,Peptides,"Amino Acids, Peptides, and Analogues",Organic Acids,"[BE0000048_inhibitor, BE0000048, Prothrombin, ..."
3,Antithrombin Proteins,Carboxylic Acids and Derivatives,DB00001,Lepirudin,Organic Compounds,Peptides,"Amino Acids, Peptides, and Analogues",Organic Acids,"[BE0000048_inhibitor, BE0000048, Prothrombin, ..."
4,Antithrombins,Carboxylic Acids and Derivatives,DB00001,Lepirudin,Organic Compounds,Peptides,"Amino Acids, Peptides, and Analogues",Organic Acids,"[BE0000048_inhibitor, BE0000048, Prothrombin, ..."


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99069 entries, 0 to 99068
Data columns (total 9 columns):
category      92910 non-null object
class         95341 non-null object
drug_id       99069 non-null object
drug_name     99069 non-null object
kingdom       95439 non-null object
parent        95439 non-null object
subclass      72040 non-null object
superclass    95439 non-null object
target        99069 non-null object
dtypes: object(9)
memory usage: 6.8+ MB


In [46]:
df['category'].fillna(df['drug_id'],inplace=True)
df['class'].fillna(df['drug_id'],inplace=True)
df['kingdom'].fillna(df['drug_id'],inplace=True)
df['parent'].fillna(df['drug_id'],inplace=True)
df['subclass'].fillna(df['drug_id'],inplace=True)
df['superclass'].fillna(df['drug_id'],inplace=True)

# nulls => drug_id because nulls are important but shouldn't be grouped together

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99069 entries, 0 to 99068
Data columns (total 9 columns):
category      99069 non-null object
class         99069 non-null object
drug_id       99069 non-null object
drug_name     99069 non-null object
kingdom       99069 non-null object
parent        99069 non-null object
subclass      99069 non-null object
superclass    99069 non-null object
target        99069 non-null object
dtypes: object(9)
memory usage: 6.8+ MB


In [48]:
list_target_action_ids = [x[0] for x in df['target']]
list_target_ids = [x[1] for x in df['target']]
list_target_names = [x[2] for x in df['target']]
list_target_actions = [x[3] for x in df['target']]

In [49]:
df_targets = pd.DataFrame({'target_action_id':list_target_action_ids,
                           'target_id':list_target_ids,
                           'target_name':list_target_names,
                           'target_action':list_target_actions})

df_targets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99069 entries, 0 to 99068
Data columns (total 4 columns):
target_action       17842 non-null object
target_action_id    99069 non-null object
target_id           99069 non-null object
target_name         99069 non-null object
dtypes: object(4)
memory usage: 3.0+ MB


In [51]:
df = df.join(df_targets)
df.drop(columns=['target','target_action_id'],inplace=True)
df['target_action'].fillna(df['drug_id'],inplace=True)
df['target_action_id'] = df['target_id'].values+'_'+df['target_action'].values
df.drop(columns=['target_id','target_name','target_action'],inplace=True)

In [63]:
df.to_pickle('pickles/df.pkl')